<a href="https://colab.research.google.com/github/asmb-09-12/Pytorch-Tutorial/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix
%matplotlib inline


In [2]:
# Convert MNIST Image giles into a Tensor of 4-Dimensions
#(# of Images, Height, Width, DN)
transform =transforms.ToTensor()

In [3]:
#Train Data
train_data = datasets.MNIST(root='/cnn_data', train =True, download = True,
                            transform = transform)

In [4]:
test_data = datasets.MNIST(root='/cnn_data', train =False, download = True,
                            transform = transform)

In [5]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [6]:
pwd

'/content'

In [7]:
cd ..

/


In [8]:
ls

bin@                        home/    NGC-DL-CONTAINER-LICENSE  srv/
boot/                       kaggle/  opt/                      sys/
cnn_data/                   lib@     proc/                     tmp/
content/                    lib32@   python-apt/               tools/
cuda-keyring_1.1-1_all.deb  lib64@   python-apt.tar.xz*        usr/
datalab/                    libx32@  root/                     var/
dev/                        media/   run/
etc/                        mnt/     sbin@


In [9]:
cd cnn_data

/cnn_data


In [10]:
ls

MNIST/


In [11]:
cd ..


/


In [12]:
cd content/

/content


In [13]:
# Create a small batch size for images
train_loader = DataLoader(train_data, batch_size = 10, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 10, shuffle = False)

In [14]:
# Define Our CNN Model
# Describe convolutional later and what it's doing (2 Convolutional layers)
conv1 =nn.Conv2d(1, 6, 3, 1)
conv2 =nn.Conv2d(6, 16, 3, 1)

In [15]:
from re import X
# Grab 1 MNIST record/ image
for i, (X_train, y_train) in enumerate(train_data):
  break

In [16]:
X_train.shape

torch.Size([1, 28, 28])

In [17]:
x = X_train.view(1,1,28,28)

In [18]:
x.shape

torch.Size([1, 1, 28, 28])

In [19]:
x = F.relu(conv1(x)) # Rectified Linear Unit for our activation function


In [20]:
x.shape

torch.Size([1, 6, 26, 26])

In [21]:
# 1 image
# 6 is the filters
# Height (start and end pixels not counted)
# Width (start and end pixels not counted)

In [22]:
x = F.max_pool2d(x, 2, 2) #kernel of 2 and stride of 2

In [23]:
x.shape

torch.Size([1, 6, 13, 13])

In [24]:
# second convolution layer

x = F.relu(conv2(x))

In [25]:
x.shape

torch.Size([1, 16, 11, 11])

In [26]:
# 1 image
# 16 is the filters
# Height (start and end pixels not counted)
# Width (start and end pixels not counted)

In [27]:
# Pooling Layer

x = F.max_pool2d(x, 2, 2)

In [28]:
x.shape

torch.Size([1, 16, 5, 5])

MODEL CLASS

In [29]:
# Model Class

In [31]:
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1 ,6 ,3 ,1)
    self.conv2 = nn.Conv2d(6, 16 ,3 ,1)
    # Fully Connected Layer i.e. Classification Layer
    self.fc1 = nn.Linear(5*5*16, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, X):
    # 1st Pass
    X = F.relu(self.conv1(X))
    X = F.max_pool2d(X, 2, 2)
    # 2nd Pass
    X = F.relu(self.conv2(X))
    X = F.max_pool2d(X, 2, 2)

    # Re-View to flatten it out
    X = X.view(-1, 5*5*16) #negative one so we can vary the batch size

    # Fully Connected Layers
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = self.fc3(X)

    return F.log_softmax(X, dim =1)

In [32]:
# Create an Instance of our Model
torch.manual_seed(41)
model1 = ConvolutionalNetwork()
model1

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [33]:
# Loss Function Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model1.parameters(), lr =0.001)
# Smaller the Learning Rate, longer its gonna take to train

In [36]:
import time
start_time = time.time()

# Create Variable to Track Thiings
epochs = 5
train_losses = []
test_losses = []
train_correct = []
test_correct = []

# For Loop of Epochs
for i in range(epochs):
  trn_corr = 0
  tst_corr = 0

  # Train
  for b, (X_train, y_train) in enumerate(train_loader):
    b += 1
    y_pred = model1(X_train)
    loss = criterion(y_pred, y_train)


    predicted = torch.max(y_pred.data, 1)[1]
    batch_corr = (predicted == y_train).sum()
    trn_corr += batch_corr



    # Update our Parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()




    # Print out some results
    if b%600 ==0:
      print(f'Epoch: {i} Batch: {b} Loss: {loss.item()}')



  train_losses.append(loss)
  train_correct.append(trn_corr)



  # Test
  with torch.no_grad():
    for b, (X_test, y_test) in enumerate(test_loader):
      y_val = model1(X_test)
      predicted = torch.max(y_val.data, 1)[1]
      tst_corr += (predicted ==y_test).sum()

  loss = criterion(y_val, y_test)
  test_losses.append(loss)
  test_correct.append(tst_corr)




current_time = time.time()
print(f'Training Time: {(current_time - start_time)/60} minutes')

Epoch: 0 Batch: 600 Loss: 0.03221464157104492
Epoch: 0 Batch: 1200 Loss: 0.003333463566377759
Epoch: 0 Batch: 1800 Loss: 0.04126342386007309
Epoch: 0 Batch: 2400 Loss: 0.14685525000095367
Epoch: 0 Batch: 3000 Loss: 0.12026345729827881
Epoch: 0 Batch: 3600 Loss: 0.0001952919119503349
Epoch: 0 Batch: 4200 Loss: 0.00016604487609583884
Epoch: 0 Batch: 4800 Loss: 0.0006250619189813733
Epoch: 0 Batch: 5400 Loss: 0.0019755663815885782
Epoch: 0 Batch: 6000 Loss: 0.0005515654920600355
Epoch: 1 Batch: 600 Loss: 0.029676983132958412
Epoch: 1 Batch: 1200 Loss: 0.0007412865525111556
Epoch: 1 Batch: 1800 Loss: 0.0008890863391570747
Epoch: 1 Batch: 2400 Loss: 0.006851988844573498
Epoch: 1 Batch: 3000 Loss: 0.3968263268470764
Epoch: 1 Batch: 3600 Loss: 0.00292785931378603
Epoch: 1 Batch: 4200 Loss: 0.0005554658127948642
Epoch: 1 Batch: 4800 Loss: 0.006729685701429844
Epoch: 1 Batch: 5400 Loss: 0.0051476918160915375
Epoch: 1 Batch: 6000 Loss: 0.0030590523965656757
Epoch: 2 Batch: 600 Loss: 0.0015550360